# CaNH2 structure including vibronic, rotational and electronic spin

### Import functions classes, define symmetry group

In [1]:
from src.group_theory.Group import *
from src.molecular_structure.VibronicStates import VibronicState, OffsetOperator
from src.molecular_structure.RotationalStates import STM_RotationalBasis
from src.molecular_structure.molecular_Hamiltonians.SpinRotation import Spin_Rotation_Hamiltonian_evCaseB
from src.quantum_mechanics.AngularMomentum import ElectronicSpinBasis
from src.quantum_mechanics.Basis import OrthogonalBasis
from src.molecular_structure.molecular_Hamiltonians.Rotational_Hamiltonian import *

# uncomment the following if you want to print matrices and not see all the digits.
# np.set_printoptions(formatter={'all': lambda x: f"{x.real:.2f}"},threshold=np.inf)


In [2]:
def wavenumber_to_Hz(k):
    if isinstance(k, list):
        return [wavenumber_to_Hz(item) for item in k]
    if isinstance(k, dict):
        return {key: wavenumber_to_Hz(k[key]) for key in k}
    return k * 299792458 * 100

def wavenumber_to_MHz(k):
    if isinstance(k, list):
        return [wavenumber_to_MHz(item) for item in k]
    if isinstance(k, dict):
        return {key: wavenumber_to_MHz(k[key]) for key in k}
    return k * 299792458 * 100 / 1e6

# define symmetry group
group = C2vGroup()
A1 = C2v_A1_representation(group)
A2 = C2v_A2_representation(group)
B1 = C2v_B1_representation(group)
B2 = C2v_B2_representation(group)

In [3]:
B1 * B2

C2v_A2 representation
e: [[1]]
C_2: [[1]]
σ_v(xz): [[-1]]
σ_v(yz): [[-1]]

### Define the vibronic basis, and their dipole moment.

Here I only included one vibrational state in each electronic state. Just add more if needed

In [4]:
from src.tools.SphericalTensors import SphericalTensor_prolate, Irreducible_SphericalTensor

X = VibronicState("X", symmetry_group=group, irrep= A1)
A = VibronicState("A", symmetry_group=group, irrep= B2)
B = VibronicState("B", symmetry_group=group, irrep= B1)
vibronic_basis = OrthogonalBasis([X,A],"Electronic")
vibronic_d_a = Operator(vibronic_basis,np.array([[0,0,0],[0,0,0],[0,0,0]]),symmetry_group=group, irrep= A1)
vibronic_d_b = Operator(vibronic_basis,np.array([[0,1,0],[1,0,0],[0,0,0]]),symmetry_group=group, irrep= B2)
vibronic_d_c = Operator(vibronic_basis,np.array([[0,0,1],[0,0,0],[1,0,0]]),symmetry_group=group, irrep= B1)

# the transition dipole moment in molecule frame as a spherical tensor
vibronic_d = SphericalTensor_prolate(np.array([vibronic_d_a,vibronic_d_b,vibronic_d_c]),is_operator=True, operator_basis=vibronic_basis)

### Define the rotational basis and electron spin basis

Note: it's important to couple vibronic and rotational states first, before coupling to other states. This is because the dipole moment operator evaluation is not separable for vibronic and rotational Hilbert spaces.

In [5]:
rot_basis = STM_RotationalBasis(R_range=(0,2))
es_basis = ElectronicSpinBasis(S_range=(1/2,1/2))
evr_basis = rot_basis * vibronic_basis

basis = evr_basis.couple(es_basis)
basis.rename_symbols("J","m") # rename angular momentum symbols.

### Define molecular constants for different electronic states

Note: all constants should be given in GHz.

In [18]:
from src.molecular_structure.RotationOperators import MShiftOperator, JShiftOperator

A_dict = {}
BC_avg2_dict = {}
BC_diff4_dict = {}

e_aa_dict = {}
e_bb_dict = {}
e_cc_dict = {}

offset_dict = {}

# X state constants
A_dict["X"] = wavenumber_to_MHz(13.05744)
BC_avg2_dict["X"] = wavenumber_to_MHz(0.296652)
BC_diff4_dict["X"] = wavenumber_to_MHz(1.8894e-3)
e_aa_dict["X"] = 45.7
e_bb_dict["X"] = 32.063
e_cc_dict["X"] = 41.110
offset_dict["X"] = 0.0


# A state constants
A_dict["A"] = wavenumber_to_MHz(11.44854)
BC_avg2_dict["A"] = wavenumber_to_MHz(0.303107)
BC_diff4_dict["A"] = wavenumber_to_MHz(1.958e-3)
e_aa_dict["A"] = wavenumber_to_MHz(8.2369)
e_bb_dict["A"] = wavenumber_to_MHz(3.0534e-2 - 1.2617e-2 * 2)
e_cc_dict["A"] = wavenumber_to_MHz(3.0534e-2 + 1.2617e-2 * 2)
offset_dict["A"] = (wavenumber_to_Hz(15464.36739)-192e6)/1e6

# B state constants
A_dict["B"] = wavenumber_to_MHz(14.3664)
BC_avg2_dict["B"] = wavenumber_to_MHz(0.301693)
BC_diff4_dict["B"] = wavenumber_to_MHz(4.68e-3)
e_aa_dict["B"] = wavenumber_to_MHz(-7.5472)
e_bb_dict["B"] = wavenumber_to_MHz(2.083e-2 - 8.66e-3 * 2)
e_cc_dict["B"] = wavenumber_to_MHz(2.083e-2 + 8.66e-3 * 2)
offset_dict["B"] = (wavenumber_to_Hz(15885.28188)-129e6)/1e6

### Define the Hamiltonian

H_rot: rotational Hamiltonian

H_SR: spin-rotation coupling

H_offset: adds offset to the energies for different electronic states

Z: breaks magnetic degeneracies by adding a fictitious term

In [19]:
H_rot = Rotational_Hamiltonian_evr(basis,A_dict,BC_avg2_dict,BC_diff4_dict)
H_SR = Spin_Rotation_Hamiltonian_evCaseB(basis, e_aa_dict, e_bb_dict, e_cc_dict)
H_offset = OffsetOperator(basis,offset_dict)
Z = MShiftOperator(basis) * 1e-3 #+ JShiftOperator(basis) * 1e-5


H = H_rot + H_SR + H_offset + Z

In [20]:
# you can check which basis states are coupled by each operator by calling
# H_SR.get_connected_states()

### Diagonalize the Hamiltonian

In [21]:
from src.molecular_structure.RotationalStates import rename_ATM_states

Es, states = H.diagonalize()
rename_ATM_states(states)  # rename the states using ATM convention (N ka kc)

## Inspect the energy eigenstates

We can take a look at some of the eigenstates. For simplicity let's restrict to the m = 1/2 magnetic states

### X State

In [22]:
X_states_m12 = []
X_Es_m12 = []
for i in range(len(states)):
    if states[i][0].m == 1/2 and states[i][0].elec == "X":
        X_states_m12.append(states[i])
        X_Es_m12.append(Es[i])

These are our cycling ground states:

In [23]:
X_states_m12[5]

|elec = X, S = 0.5, N=1, ka=1, kc=1, J = 0.5, m = 0.5> = 0.71 |elec=X, S=0.5, N=1, k=-1, J=0.5, m=0.5> + -0.71 |elec=X, S=0.5, N=1, k=1, J=0.5, m=0.5> + -2.0e-05 |elec=X, S=0.5, N=1, k=-1, J=0.5, m=-0.5> + 2.0e-05 |elec=X, S=0.5, N=1, k=1, J=0.5, m=-0.5> 

In [24]:
X_states_m12[6]

|elec = X, S = 0.5, N=1, ka=1, kc=1, J = 1.5, m = 0.5> = 0.71 |elec=X, S=0.5, N=1, k=1, J=1.5, m=0.5> + -0.71 |elec=X, S=0.5, N=1, k=-1, J=1.5, m=0.5> + 3.9e-05 |elec=X, S=0.5, N=2, k=-1, J=1.5, m=0.5> + 3.9e-05 |elec=X, S=0.5, N=2, k=1, J=1.5, m=0.5> + 3.1e-05 |elec=X, S=0.5, N=1, k=1, J=1.5, m=-0.5> + -3.1e-05 |elec=X, S=0.5, N=1, k=-1, J=1.5, m=-0.5> + 1.4e-05 |elec=X, S=0.5, N=1, k=1, J=1.5, m=1.5> + -1.4e-05 |elec=X, S=0.5, N=1, k=-1, J=1.5, m=1.5> 

In [25]:
print(f"1_11 state spin-rotation splitting = {(X_Es_m12[6] - X_Es_m12[5]):.1f} MHz")

1_11 state spin-rotation splitting = 65.1 MHz


These are the 2_11 states:

In [15]:
X_states_m12[11]

|elec = X, S = 0.5, N=2, ka=1, kc=1, J = 1.5, m = 0.5> = 0.71 |elec=X, S=0.5, N=2, k=1, J=1.5, m=0.5> + 0.71 |elec=X, S=0.5, N=2, k=-1, J=1.5, m=0.5> + 3.9e-05 |elec=X, S=0.5, N=1, k=-1, J=1.5, m=0.5> + -3.9e-05 |elec=X, S=0.5, N=1, k=1, J=1.5, m=0.5> 

In [16]:
X_states_m12[12]

|elec = X, S = 0.5, N=2, ka=1, kc=1, J = 2.5, m = 0.5> = 0.71 |elec=X, S=0.5, N=2, k=1, J=2.5, m=0.5> + 0.71 |elec=X, S=0.5, N=2, k=-1, J=2.5, m=0.5> 

In [17]:
print(f"2_11 state spin-rotation splitting = {(X_Es_m12[12] - X_Es_m12[11])*1000:.1f} MHz")

2_11 state spin-rotation splitting = 89.6 MHz


### A State

In [18]:
A_states_m12 = []
A_Es_m12 = []
for i in range(len(states)):
    if states[i][0].m == 1/2 and states[i][0].elec == "A":
        A_states_m12.append(states[i])
        A_Es_m12.append(Es[i])

This is the A 0_00 J=1/2 state, which is the excited state in our cycling scheme:

In [19]:
A_states_m12[0]

|elec = A, S = 0.5, N=0, ka=0, kc=0, J = 0.5, m = 0.5> = 1.00 |elec=A, S=0.5, N=0, k=0, J=0.5, m=0.5> 

In [20]:
print(f"Cycling transition frequency on X-A: {(A_Es_m12[0] - X_Es_m12[5])*1e-3:.6f} THz")

Cycling transition frequency on X-A: 463.209690 THz


### B State

In [21]:
B_states_m12 = []
B_Es_m12 = []
for i in range(len(states)):
    if states[i][0].m == 1/2 and states[i][0].elec == "B":
        B_states_m12.append(states[i])
        B_Es_m12.append(Es[i])

This is the B 0_00 state, which is the excited state for X-B cycling scheme:

In [22]:
B_states_m12[0]

IndexError: list index out of range

For completeness, these are the ground states for the X-B cycling scheme:

In [ ]:
X_states_m12[7]

In [ ]:
X_states_m12[8]

In [ ]:
print(f"Cycling transition frequency on X-B: {(B_Es_m12[0] - X_Es_m12[7])*1e-3:.6f} THz")

Let's also look at the B 1_01 SR states (excited states for our 6_2 repump):

In [ ]:
B_states_m12[1]

In [ ]:
B_states_m12[2]

In [ ]:
print(f"Spin-rotation on B 1_01: {(B_Es_m12[2] - B_Es_m12[1]):.6f} GHz")

### Optional: decompose Hund's case B into STM basis

If you are curious about the decomposition into rotation x spin, you can do it like this:

In [ ]:
STM_basis = basis.get_uncoupled_basis()
matrix = basis.get_basis_change_matrix()
s = X_states_m12[5].change_basis(STM_basis,matrix)
s

# Transition Dipole Moments

Define the dipole operator on the vibronic x rotational Hilbert space and on the electron spin Hilbert space. Take their tensor product.

sigma_d is the spatial angular momentum of the dipole moment. So sigma_d = 0 is looking at the pi transitions.

In [ ]:
from src.molecular_structure.TDMs import DipoleOperator_evr, DipoleOperator_spin

TDM = Irreducible_SphericalTensor(1,is_operator=True,operator_basis=basis)

for sigma_d in (-1,0,1):
    d_evr = DipoleOperator_evr(evr_basis,vibronic_d, 0)
    d_es = DipoleOperator_spin(es_basis,0)
    d_uncoupled = d_evr.tensor(d_es)
    d = d_uncoupled.change_basis(basis, basis.get_basis_change_matrix())
    TDM[sigma_d] = d

For example, check the dipole moment between X 1_11 J=3/2, m=1/2 and A 0_00 J=1/2, m=1/2:

In [ ]:
d = TDM[0]
s1 = X_states_m12[6]
s2 = A_states_m12[0]
print(f"{d.matrix_element(s1, s2):.2f}")

And check the dipole moment between X 1_10 J=3/2, m=1/2 and A 0_00 J=1/2, m=1/2: (it should be zero, because X 1_01 has the same parity as A 0_00)

In [ ]:
s1 = X_states_m12[8]
s2 = A_states_m12[0]
print(f"{d.matrix_element(s1, s2):.2f}")

It is, instead, connected to B 0_00 state:

In [ ]:
s1 = X_states_m12[8]
s2 = B_states_m12[0]
print(f"{d.matrix_element(s1, s2):.2f}")

Look at dipole-connected Case B basis states using d.get_connected_states()

In [ ]:
connected_states = d.get_connected_states()
connected_states

### Zeeman Shifts

In [23]:
from src.molecular_structure.Zeeman import ZeemanHamiltonian_es

Zeeman_z_evr = ZeemanHamiltonian_es(evr_basis, "z")
Zeeman_z_es = ZeemanHamiltonian_es(es_basis,"z")
Zeeman_z_uncoupled = Zeeman_z_evr.tensor(Zeeman_z_es)
Zeeman_z = Zeeman_z_uncoupled.change_basis(basis, basis.get_basis_change_matrix())


In [24]:
np.max(Zeeman_z.matrix)

np.complex128(0.00140124805611607+0j)

In [25]:
basis[0].show_composition()

|elec=X, S=0.5, N=0, k=0, J=0.5, m=-0.5> = 1.00 |elec=X, S=0.5, ms=-0.5, N=0, k=0, mN=0> 


In [26]:
Zeeman_z[basis[0],basis[0]]

np.complex128(-0.00140124805611607+0j)